In [ ]:
from urllib.request import urlretrieve # 이미지 파일 저장 용도
import requests # url 접속 용도

from bs4 import BeautifulSoup

In [ ]:
url = "https://store.musinsa.com/app/goods/2064478"

response = requests.get(url)
print(response)

<Response [403]>


In [ ]:
from fake_useragent import UserAgent
user_agent = {'User-agent': UserAgent().chrome} # 나는 파이썬이 아니고 Mozilla 라는 브라우져야! 라고 속이는 것. 이 기법이 항상 통하는 것은 아님..
response  = requests.get(url, headers = user_agent)
print(response)

<Response [200]>


In [ ]:
product_page = response.content
product_soup = BeautifulSoup(product_page, 'html.parser')
product_soup

<!DOCTYPE html>

<html lang="ko">
<head>
<!-- Google Tag Manager -->
<script>
var selfCertEnabled = "" || "Y";
var THIS_PAGE_GF = 'A'; // 전역필터 상태값 caching page reload 필요 체크용
var dataLayer = window.dataLayer || [];
window.addEventListener("DOMContentLoaded", function() {
    
    (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
            new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
        j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
        'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
    })(window,document,'script','dataLayer','GTM-TXDSFSF');
}, false);
</script>
<!-- End Google Tag Manager -->
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="/favicon.ico?20200213" rel="shortcut icon" type="image/x-icon"/>
<meta content="제품분류 : 아우터 &gt; 사파리/헌팅 재킷 브랜드 : 라퍼지스토어(LAFUDGESTORE) 제품번호 : fudge 제품 : [리뉴얼 ver.][3M 신슐레이트]오리지널 M-1965 피쉬테일 파카_Original Kh

스크래이핑 할 때는 html 페이지의 패턴 파악 부터!!

# 이미지 크롤링

In [ ]:
# 이미지 썸네일 칸 데이터 가져오기
thumb_imgs = product_soup.select("#detail_thumb > ul > li > img")
len(thumb_imgs)

1

In [ ]:
img_url_splits = thumb_imgs[0]['src'].split("/")
img_url_splits[-1] = img_url_splits[-1].replace("_60", "_500")
print("https:" + '/'.join(img_url_splits))

https://image.msscdn.net/images/goods_img/20210813/2064478/2064478_3_500.jpg


In [ ]:
# img_url_splits = thumb_imgs[2]['src'].split("/")
# img_url_splits[-1] = img_url_splits[-1].replace("_60", "_500")
# print("https:" + '/'.join(img_url_splits))

IndexError: list index out of range

In [ ]:
for thumb_img in thumb_imgs:
  img_url_splits = thumb_img['src'].split("/")
  img_url_splits[-1] = img_url_splits[-1].replace("_60", "_500")
  product_img_url = "https:" + '/'.join(img_url_splits)
  filename = product_img_url.split("/")[-1]
  urlretrieve(product_img_url, filename)

다른 상품 페이지도 잘 크롤링이 될까?

In [ ]:
import os # 폴더를 만들기 위해 os 패키지 import

url = "https://store.musinsa.com/app/goods/2208254"

user_agent = {'User-agent': UserAgent().chrome} # 나는 파이썬이 아니고 Mozilla 라는 브라우져야! 라고 속이는 것. 이 기법이 항상 통하는 것은 아님..
response  = requests.get(url, headers = user_agent)

product_page = response.content
product_soup = BeautifulSoup(product_page, 'html.parser')

thumb_imgs = product_soup.select("#detail_thumb > ul > li > img")

# 폴더 만들기
dir_name = "./{}".format(url.split("/")[-1])
os.mkdir(dir_name)

for thumb_img in thumb_imgs:
  img_url_splits = thumb_img['src'].split("/")
  img_url_splits[-1] = img_url_splits[-1].replace("_60", "_500")
  product_img_url = "https:" + '/'.join(img_url_splits)
  filename = product_img_url.split("/")[-1]
  urlretrieve(product_img_url, "{}/{}".format(dir_name, filename))

# 상품 정보 크롤링

In [ ]:
# 브랜드 / 상품 코드
product_info_list = product_soup.select("ul.product_article > li")
product_brand = product_info_list[0].select_one("p.product_article_contents > strong").text.split(" / ")[0]
product_code = product_info_list[0].select_one("p.product_article_contents > strong").text.split(" / ")[1]

print(product_brand, product_code)

GIORDANO 071924


In [ ]:
# 시즌, 성별
import re
product_season_gender = product_info_list[1].select_one("p.product_article_contents").text.strip()
product_season_gender = re.sub("[ ]{2,}", " ", product_season_gender).split("\n/\n")

product_season = product_season_gender[0].replace("\n", "")
product_gender = product_season_gender[1].replace("\n", "")

print(product_season, product_gender)

2021 F/W  남


In [ ]:
# 좋아요 수
# product_info_list[4].select_one("span.prd_like_cnt").text
import json

user_agent = {
    'User-agent': UserAgent().chrome,
    "content-type": "application/json", # json으로 요청을 할거야~
}
counts = requests.post("https://like.musinsa.com/like/api/v1/liketypes/goods/counts",
                       
                       # json.dumps : 딕셔너리(dict)를 json화 시켜준다.
                       data=json.dumps({"relationIds":["751030"]}), # data : parameter 전달 하기.. json type으로 파라미터를 전달.
                       headers = user_agent)

data = json.loads(counts.content) # json을 딕셔너리(dict)화 시켜준다.
data['data']['contents'][0]['count']

24723

In [ ]:
product_info_list[4].select_one("span.prd_like_cnt").text

'0'

In [ ]:
product_id = 2208254
page = 1
# 사진 후기
review_photo_url = "https://store.musinsa.com/app/reviews/goods_estimate_list/photo/{}/0/{}".format(product_id, page)
review_photo_page = requests.get(review_photo_url, headers=user_agent).content
review_photo_soup = BeautifulSoup(review_photo_page, 'html.parser')
review_photo_soup.select("div.review-contents__text")[1].text.strip()

'따뜻하고 이쁘고 핏도 예쁘고 크기도 적당하고 좋아요'

In [ ]:
product_id = 2208254
page = 1
# 스타일 후기
review_style_url = "https://store.musinsa.com/app/reviews/goods_estimate_list/style/{}/0/{}".format(product_id, page)
review_style_page = requests.get(review_style_url, headers=user_agent).content
review_style_soup = BeautifulSoup(review_style_page, 'html.parser')
review_style_soup.select("div.review-contents__text")[1].text.strip()

'색상도 맘에들고 보온성도 좋아요 잘입을거같아요'

In [ ]:
product_id = 2208254
page = 2
# 상품 후기
review_goods_url = "https://store.musinsa.com/app/reviews/goods_estimate_list/goods/{}/0/{}".format(product_id, 1)
review_goods_page = requests.get(review_goods_url, headers=user_agent).content
review_goods_soup = BeautifulSoup(review_goods_page, 'html.parser')
review_goods_soup.select("div.review-contents__text")[1].text.strip()

'색이 진짜 예쁘고 흔하지도 않아서 졸습니다 추천해요'